In [ ]:
import pickle

l = []
for i in range(len(gold_emotions)):
  if gold_emotions[i] != predicted_emotions[i]:
    l.append(i)
  
s = []
for i in l:
  s.append(outs.predictions[0][i])

# open a file, where you ant to store the data
file = open('logist_errate.pkl', 'wb')

# dump information to that file
pickle.dump(s, file)

# close the file
file.close()

In [1]:
import pandas as pd
path = 'logist_errate.pkl'
obj = pd.read_pickle(path)

In [3]:
obj[:10]

[array([-8.064367 , -7.201281 , -6.8728104, -5.240647 , -6.4233584,
         0.7787335,  1.9870163, -7.222992 ], dtype=float32),
 array([-6.944066 ,  1.7112933, -6.1993055, -6.051587 , -6.8772316,
        -1.3442721, -3.657677 , -5.2473454], dtype=float32),
 array([-4.6992183, -4.982564 , -6.7400994, -6.209856 , -6.613549 ,
         4.45661  , -6.5877223, -6.761219 ], dtype=float32),
 array([-4.8130465, -4.5439067, -3.2511902, -6.1554418, -6.4552383,
        -7.0971675,  6.6741643, -6.376077 ], dtype=float32),
 array([-5.1539335, -6.333181 , -5.293275 , -7.0705185, -7.1192274,
        -4.202336 ,  5.933602 , -7.339711 ], dtype=float32),
 array([-3.8355594,  4.414359 , -5.0458574, -5.4942336, -5.8431177,
        -6.23308  ,  1.7687093, -5.4080048], dtype=float32),
 array([-6.409288 , -6.191746 , -5.83625  , -5.684992 , -6.1039276,
         4.9795356, -5.9744163, -6.1504884], dtype=float32),
 array([-6.221209 , -6.0350413, -5.4647164, -6.5888824, -6.9954085,
        -6.434654 ,  5.80316 

In [6]:
import pandas as pd
import numpy as np
from nrclex import NRCLex
import nltk

nltk.download('punkt')

def generate_prompt(essay, gender, education, ethnicity, age, income, empathy, distress):
    if gender == 1: gender_str = "male"
    else: gender_str = "female"

    if education == 1: education_str = "with less than a high school diploma"
    elif education == 2: education_str = "with a high school diploma"
    elif education == 3: education_str = "went to a technical/vocational school"
    elif education == 4: education_str = "went to college"
    elif education == 5: education_str = "with a two year associate degree"
    elif education == 6: education_str = "with a four year bachelor's degree"
    else: education_str = "postgradute or with a professional degree"

    if ethnicity == 1: ethnicity_str = " white"
    elif ethnicity == 2: ethnicity_str = " hispanic or latino"
    elif ethnicity == 3: ethnicity_str = " black or african american"
    elif ethnicity == 4: ethnicity_str = " native american or american indian"
    elif ethnicity == 5: ethnicity_str = " asian/pacific islander"
    else: ethnicity_str = ""

    text_prompt_bio = "An essay written by a {} years old{} {}, {}, with an income of {}$.".format(
        age, ethnicity_str,
        gender_str,
        education_str,
        income
        )
    
    if empathy is not None:
        if empathy < 3: empathy_value = "low"
        elif empathy < 5: empathy_value = "medium"
        else: empathy_value = "high"
        if distress < 3: distress_value = "low"
        elif distress < 5: distress_value = "medium"
        else: distress_value = "high"
        text_prompt_emp = "The essay expresses {} empathy and {} distress levels.".format(
            empathy_value,
            distress_value
            )

    emotions = NRCLex(essay).top_emotions
    if (sum(np.array([emo[1] for emo in emotions])))==0:
        emotions = {'neutral': 1}
    n_emo = len(emotions)
    emo_string = ""
    for i, emo in enumerate(emotions):
        emo_string += emo[0]
        if i < n_emo-1:
            emo_string += ", "
    text_prompt_emo = " The top emotions expressed in the essay are: {}.".format(emo_string)
    
    return text_prompt_bio, text_prompt_emp, text_prompt_emo

def add_prompt(dataframe):
    dataframe["prompt_bio"] = ""
    dataframe["prompt_emp"] = ""
    dataframe["prompt_emo"] = ""
    for idx, row in dataframe.iterrows():
        bio_prompt, emp_prompt, emo_prompt = generate_prompt(
            row['essay'],
            row['gender'],
            row['education'],
            row['race'],
            row['age'],
            row['income'],
            row['empathy'] if 'empathy' in row else None,
            row['distress'] if 'empathy' in row else None
            )
        dataframe.at[idx, "prompt_bio"] = bio_prompt
        dataframe.at[idx, "prompt_emp"] = emp_prompt
        dataframe.at[idx, "prompt_emo"] = emo_prompt
    return dataframe

year=23

TRAIN = f"datasets/WASSA{year}_essay_level_original_internal_train_preproc.tsv"
VAL = f"datasets/WASSA{year}_essay_level_original_internal_val_preproc.tsv"
DEV = f"datasets/WASSA{year}_essay_level_dev_preproc.tsv"

train_df = pd.read_csv(TRAIN, sep='\t')
val_df = pd.read_csv(VAL, sep='\t')
dev_df = pd.read_csv(DEV, sep='\t')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/irenetesta/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
new_train = add_prompt(train_df)
new_train.to_csv(f"datasets/WASSA{year}_essay_level_original_internal_train_preproc_prompt.tsv", index=False, sep='\t')
new_val = add_prompt(val_df)
new_val.to_csv(f"datasets/WASSA{year}_essay_level_original_internal_val_preproc_prompt.tsv", index=False, sep='\t')
new_dev = add_prompt(dev_df)
new_dev.to_csv(f"datasets/WASSA{year}_essay_level_dev_preproc_prompt.tsv", index=False, sep='\t')

